<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Tarea 1 Analisis Numerico</i></h1>
  </font>
  <font color="Blue" face="Comic Sans MS,arial">
  <h5 align="center"><i>MARTINEZ ROSAS ZAYDE YAMILE</i></h5>
  <h5 align="center"><i>CAMACHO MARIN ANA KAREN</i></h5>
  <h5 align="center"><i>LÓPEZ AGUIRRE ROBERTO OCELOTZIN</i></h5>
  <h5 align="center"><i>VARGAS BAUTISTA EMMANUEL</i></h5>
  </font>

In [ ]:
import numpy as np

# ---------- Datos dados ---------- #
m, n = 21, 12
epsilon = 1e-10

# Usamos la funcion np.linspace para ayudarnos a generar datos de una variable continua uniforme en el [0,1]
t = np.linspace(0, 1, m)

# Con esta funcion de numpy nos ayudamos a generar una funcion tal y como se propone es decir de columnas crecientes
A = np.vander(t, N=n, increasing=True)

# Funcion de numpy que hemos usado en numerosas ocaciones que genera una matriz identidad
x_exact = np.ones(n)

# Calculamos los y_i = A @ x_exact
y = A @ x_exact

# Hacemos las pertubaciones respectivas
u = np.random.rand(m)
y_perturbed = y + (2 * u - 1) * epsilon

# ---------- Método de Cholesky (Ecuaciones Normales) ---------- #

# Ecuaciones normales
ATA = A.T @ A
ATy = A.T @ y_perturbed

# Cholesky: ATA = L @ L.T (L triangular inferior)

def cholesky_jamon(A):
    n = A.shape[0]
    L = np.zeros_like(A)

    for i in range(n):
        for j in range(i + 1):
            sum_k = sum(L[i][k] * L[j][k] for k in range(j))

            if i == j:  # Diagonal
                val = A[i][i] - sum_k
                if val <= 0:
                    raise ValueError("La matriz no es definida positiva.")
                L[i][j] = np.sqrt(val)
            else:  # Fuera de la diagonal
                L[i][j] = (A[i][j] - sum_k) / L[j][j]

    return L

L = cholesky_jamon(ATA)

#Procedemos a implementar la sustitucion hacia delante y hacia atras visto en las tareas anteriores

def sustitucion_hacia_delante(L, b):
    n = L.shape[0]
    z = np.zeros_like(b)
    for i in range(n):
        z[i] = (b[i] - np.dot(L[i, :i], z[:i])) / L[i, i]
    return z

def sustitucion_hacia_delante(U, b):
    n = U.shape[0]
    x = np.zeros_like(b)
    for i in reversed(range(n)):
        x[i] = (b[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x


#Aplicamos lo programado anteriormente

z = sustitucion_hacia_delante(L, ATy)

x_cholesky = sustitucion_hacia_delante(L.T, z)

# ---------- Método de QR ----------

def qr(A):
    m, n = A.shape
    R = A.copy().astype(float)
    Q = np.eye(m)

    for k in range(n):
        # Selecciona la subcolumna desde fila k hacia abajo
        x = R[k:, k]
        e = np.zeros_like(x)
        e[0] = np.linalg.norm(x)
        if x[0] < 0:
            e[0] *= -1

        u = x - e
        v = u / np.linalg.norm(u)

        # Construye la matriz de Householder H = I - 2vvᵗ
        Hk = np.eye(m)
        Hk_k = np.eye(len(v)) - 2.0 * np.outer(v, v)
        Hk[k:, k:] = Hk_k

        R = Hk @ R
        Q = Q @ Hk

    # Recorta Q para que sea m×n (modo 'económico')
    Q = Q[:, :n]
    R = R[:n, :]

    return Q, R


Q, R = qr(A)

QTy = Q.T @ y_perturbed

x_qr = sustitucion_hacia_delante(R, QTy)

# ---------- Comparaciones pedidas de manera numerica ----------

# Errores en coeficientes
error_cholesky = np.linalg.norm(x_cholesky - x_exact)
error_qr = np.linalg.norm(x_qr - x_exact)

# Ajustes a los datos
y_fit_cholesky = A @ x_cholesky
y_fit_qr = A @ x_qr

# Errores residuales
residual_cholesky = np.linalg.norm(y_perturbed - y_fit_cholesky)
residual_qr = np.linalg.norm(y_perturbed - y_fit_qr)

# Printemos todo

print("Error en x (Cholesky):", error_cholesky)
print("Error en x (QR):      ", error_qr)
print("Residual (Cholesky):  ", residual_cholesky)
print("Residual (QR):        ", residual_qr)


Error en x (Cholesky): 376904560079431.7
Error en x (QR):       0.0024660479019129274
Residual (Cholesky):   17427997.287877902
Residual (QR):         1.514749762467485e-10


Resolvemos con NUMPY :)


In [ ]:
import numpy as np
from scipy.linalg import cho_factor, cho_solve, qr

# Parámetros
m = 21
n = 12
epsilon = 1e-10
np.random.seed(42)  # para reproducibilidad

# Datos base
t = np.linspace(0, 1, m)
V = np.vander(t, N=n, increasing=True)  # matriz de Vandermonde
x_true = np.ones(n)
y_exact = V @ x_true

# Perturbación
u = np.random.rand(m)
y = y_exact + (2 * u - 1) * epsilon

# Método 1: Ecuaciones normales + Cholesky
A = V
ATA = A.T @ A
ATy = A.T @ y
c, lower = cho_factor(ATA)
x_cholesky = cho_solve((c, lower), ATy)

# Método 2: Factorización QR
Q, R = qr(A, mode='economic')
x_qr = np.linalg.solve(R, Q.T @ y)

# Comparaciones
error_cholesky = np.linalg.norm(x_cholesky - x_true)
error_qr = np.linalg.norm(x_qr - x_true)

ajuste_cholesky = A @ x_cholesky
ajuste_qr = A @ x_qr
ajuste_true = A @ x_true

residuo_cholesky = np.linalg.norm(ajuste_cholesky - y)
residuo_qr = np.linalg.norm(ajuste_qr - y)

print("Error Cholesky:", error_cholesky)
print("Error QR:", error_qr)

Error Cholesky: 1.7311234970346405
Error QR: 0.0008652807660481151




# a) ¿Para cuál de los métodos la solución es más sensible a la perturbación generada?

La solución obtenida por Ecuaciones Normales con Cholesky es más sensible a la perturbación. Esto ocurre porque el sistema normal $A^T A$ cuadra la condición de $A$, es decir, su número de condición es el cuadrado del de $A$, lo que amplifica los errores.

# b) ¿Cuál de los métodos está más próximo a tener la solución exacta dada por $x_i = 1$ ?

El método de Factorización QR está mucho más cerca de la solución exacta. Su error relativo respecto a los coeficientes reales $x_i = 1$ es mucho menor que el error del método de Cholesky

# c) ¿La diferencia en las soluciones afecta en el ajuste de puntos $(t_i, y_i)$ por el polinomio, por qué?

No significativamente. Ambos métodos ajustan muy bien los puntos porque minimizan la misma función de error:

$$\min_x \|Ax - y\|_2$$

Entonces, aunque los coeficientes $x$ obtenidos difieren, el ajuste del polinomio es prácticamente el mismo.
Esto muestra que **múltiples conjuntos de coeficientes pueden producir ajustes igualmente buenos** cuando el sistema es mal condicionado.
